# Language Model Training Pipeline

*Currently only tested for GPT2!* 

#### Imports

In [1]:
import glob
import json
import time
# import evaluate
import transformers
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from datasets import Dataset
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence, Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.decoders import BPEDecoder
from transformers import AutoConfig, \
    DataCollatorWithPadding, AutoModelForSequenceClassification, \
    Trainer, TrainingArguments, AutoTokenizer, GPT2Config
from matplotlib import pyplot as plt
import torch

### Config

In [2]:
data_path = Path("/home/ajain/ttmp/PBSCSR_data/")
'''Path to large data folder'''

gpt2_dir = data_path/"gpt2"

seed = 42
'''Random seed: int'''

train_file = gpt2_dir/"LM_pretraining_data/train.txt"
train_file.parent.mkdir(exist_ok=True)
valid_file = gpt2_dir/"LM_pretraining_data/valid.txt"

cache_dir = data_path/".cache"
cache_dir.mkdir(exist_ok=True)
'''Path to save cache'''

# labeled_data_path = "../../100_way_dataset.zip"  # TODO: FILL IN AFTER UNZIPPING
# '''Path to labeled data'''

'Path to save cache'

### Language Model Full Finetuning Classifier

Fill in the cell below

Prewritten Helper Functions & Classes

In [3]:
def top_x_acc(y_true, y_pred, x):
    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    ranked = torch.argsort(y_pred, axis=-1)
    top_x = ranked[..., -x:]
    return (top_x == torch.repeat_interleave(y_true.unsqueeze(-1), x, axis=-1)).float().sum(-1).mean().item()

def mean_recip_rank(y_true, y_pred):
    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    ranked = torch.argsort(y_pred, axis=-1)
    true_ranks = y_pred.shape[-1] - (ranked == torch.repeat_interleave(y_true.unsqueeze(-1), y_pred.shape[-1], axis=-1)).float().argmax(-1)

    return (1/true_ranks).mean().item()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = np.argmax(predictions, axis=1)

    top_one = top_x_acc(labels, predictions, 1)
    top_five = top_x_acc(labels, predictions, 5)
    top_ten = 0
    if predictions.shape[-1] >= 10:
        top_ten = top_x_acc(labels, predictions, 10)
    mrr = mean_recip_rank(labels, predictions)

    metrics = {"top_one" : top_one, "top_five" : top_five, "mrr": mrr, "top_ten": top_ten}
    return metrics

In [4]:
print(gpt2_dir)

/home/ajain/ttmp/PBSCSR_data/gpt2


In [5]:

def process_filler(filler_file, imslp_bootleg_path, filler_threshold=0.5):
    composer_dict = {}
    with open(filler_file, "r") as fin:
        lines = fin.readlines()
        lines = [line.split("\t") for line in lines]
        for path, page, score in lines:
            parts = path.split("/")
            composer, piece, id = parts[0], "/".join(parts[1:-1]), parts[-1]
            composer_dict[composer] = {} if composer not in composer_dict else composer_dict[composer]
            composer_dict[composer][piece] = {} if piece not in composer_dict[composer] else composer_dict[composer][piece]
            composer_dict[composer][piece][id] = {"valid_pages":[], "count":0, "bscore": []} if id not in composer_dict[composer][piece] else composer_dict[composer][piece][id]
            if float(score) < filler_threshold:
                bscore_page = pd.read_pickle(imslp_bootleg_path/f"{path}.pkl")[int(page)]
                composer_dict[composer][piece][id]["valid_pages"].append(int(page))
                composer_dict[composer][piece][id]["count"] += len(bscore_page)
                composer_dict[composer][piece][id]["bscore"].append(bscore_page)
    return composer_dict


def ints_to_binary_matrix(score_seq):  # converts integer sequence to n x 62 matrix
    matrix = []
    for event in score_seq:
        binary_rep = list(np.binary_repr(event, 62))
        matrix.append(binary_rep)
    np_mat = np.array(matrix, dtype=np.uint8)
    #np_mat = np.flip(np_mat, axis=0)  # flip to have least significant bit at the front
    return np_mat


def create_dataset(pieces, valid_split=.15, test_split=.15):
    """
    Creates a train / valid / test split dataset of pieces.
    pieces: The list of binary_matrices to sample from
    valid_split: The proportion of data to use for valid
    test_split: The proportion of data to use for valid
    
    returns:
    x & y lists for train, valid, and test sets
    """
    
    # For repeatability
    np.random.seed(42)

    # shuffle pieces
    piece_list = [piece for piece in pieces]
    np.random.shuffle(piece_list)
    
    # Calculate starting places of each section - order is (test, valid, train)
    train_start = round((valid_split+test_split)*len(piece_list))
    valid_start = round(test_split*len(piece_list))
    
    # Go through and separate pieces into train, valid, test
    train_pieces = piece_list[train_start:]
    valid_pieces = piece_list[valid_start:train_start]
    test_pieces = piece_list[:valid_start]
    
    return train_pieces, valid_pieces, test_pieces

def merge_staff_overlaps(bscores):
    """
    Takes in either one binary score or a batch of them and merges the left and right hands
    """
    
    # Lower middle c is index 23
    # Upper middle c is index 33
    lower = 23
    upper = 33
    middle = (lower + upper) // 2
    
    # Total notes is 52
    total = 52
    
    # Pad out upper hand and lower hand and combine them
    padded_lower = np.concatenate([bscores[..., :middle], np.zeros((*bscores.shape[:-1], total-middle))], axis=-1)
    padded_upper = np.concatenate([np.zeros((*bscores.shape[:-1], middle-bscores.shape[-1]+total)), bscores[..., middle:]], axis=-1)
    # Logical or
    merged = padded_lower + padded_upper - padded_lower * padded_upper
    return merged


# Dense Encoder

# Continuous line of 256 unicode characters
start = 10060# 931
dense_characters = [chr(i).encode("utf-8").decode("utf-8") for i in range(start, start+512)]


# This code divides the fragment into blocks (and discards any remaining info at the very edges)
# Then it uses einsum with a filter of powers of 2 to convert from binary to an integer.  Then converts integers into
# unicode characters

def dense_encoder(fragment, block_size=[1, 1]):
    fragment = merge_staff_overlaps(fragment)
    # Rewrote this to be much faster but looks complicated
    # This filter has powers of 2 which is how the binary is turned to ints
    filter_ = np.power(2, np.arange(np.prod(block_size))).reshape(block_size)
    
    # The fragment is split into blocks here
    xblocks = np.stack(np.split(fragment[:, :(fragment.shape[1]//block_size[1])*block_size[1]], fragment.shape[1]//block_size[1], axis=1))
    xyblocks = np.stack(np.split(xblocks[:, :(xblocks.shape[1]//block_size[0])*block_size[0]], xblocks.shape[1]//block_size[0], axis=1))
    
    # The blocks are multiplied so they are ints
    numbers = np.einsum("ijkl,kl->ij", xyblocks, filter_)
    
    # The ints are turned into corresponding characters
    characters = (numbers+start).astype(np.int32).view('U1')
    return " ".join(["".join(t) for t in characters])

def data_preparation(labeled_data):
    train_X, train_y, val_X, val_y, test_X, test_y, train_m, valid_m, test_m = pd.read_pickle(labeled_data)

    train_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in train_X], "label": train_y})
    val_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in val_X], "label": val_y})
    test_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in test_X], "label": test_y})
    
    return train_df, val_df, test_df

def label2id_function(examples, label2id):
    return {"label": [label2id[label] for label in examples["label"]]}

def tokenizer_function(examples, tokenizer):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [6]:
vocab_size = 30_000

pretrained_output = "../../../PBSCSR_data/gpt2/classifier_100_pretrained/checkpoint-13128"

load_pretrained_weights = True

config_class = GPT2Config
'''Config class for language model: e.g. GPT2Config'''

lm_config = {
    'model_type': 'gpt2', # e.g. 'gpt2',
    'vocab_size': vocab_size, # e.g. 50257,
    'n_positions': 1024, # e.g. 1024,
    'n_layer': 6, # e.g. 12,
    # add more config here if needed
}

Training Classifier

In [8]:
batch_size = 8
'''Batch size for classifier training: int'''
epochs = 3
'''Epochs for classifier training: int'''
learning_rate = 5e-5
'''Learning rate for classifier training: float e.g. 5e-5'''
classifier_output_model_path = "../../../PBSCSR_data/gpt2/100_way_LP_FT"

# Prepare data
# !unzip -o {labeled_data_path} -d {data_path} 
labeled_data = data_path/"100_way_dataset.pkl"
train_df, val_df, test_df = data_preparation(labeled_data)

train_ds = Dataset.from_dict(train_df)
val_ds = Dataset.from_dict(val_df)
test_ds = Dataset.from_dict(test_df)

# Define label map
label2id = {label: i for i, label in enumerate(set(train_df['label']))}
id2label = {i: label for label, i in label2id.items()}

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_output)# , return_token_type_ids=False)
tokenizer.pad_token = '<pad>'

def tokenizer_function(examples, tokenizer):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

# Define model
if load_pretrained_weights:
    config = AutoConfig.from_pretrained(pretrained_output)
    config.num_labels = len(label2id)
    config.pad_token_id = tokenizer.pad_token_id
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_output, config=config).to("cuda")
else:
    config = config_class(**lm_config)
    config.num_labels = len(label2id)
    config.pad_token_id = tokenizer.pad_token_id
    model = AutoModelForSequenceClassification.from_config(config).to("cuda")
tokenizer.model_max_length = config.n_positions

model.resize_token_embeddings(len(tokenizer))

# Tokenize and convert labels to ids
train_ds = train_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
val_ds = val_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
test_ds = test_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
train_ds = train_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})
val_ds = val_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})
test_ds = test_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest", max_length=1024)

# Freeze all layers except the classifier
# NOT FREEZING ANYMORE

# for name, param in model.named_parameters():
#     param.requires_grad = False
# model.score.weight.requires_grad = True

# Define training arguments
training_args = TrainingArguments(
    output_dir=classifier_output_model_path,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()
trainer.evaluate(test_ds)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/abunn/ttmp/miniconda3/envs/EWLLMs/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Top One,Top Five,Mrr,Top Ten
1,3.521100,4.083024,0.126933,0.316000,0.232362,0.451867
2,1.934300,4.523793,0.125000,0.319600,0.231612,0.450467
3,0.956100,4.927619,0.125600,0.309867,0.228732,0.442667


/home/abunn/ttmp/miniconda3/envs/EWLLMs/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/abunn/ttmp/miniconda3/envs/EWLLMs/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/abunn/ttmp/miniconda3/envs/EWLLMs/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 4.1455230712890625,
 'eval_top_one': 0.12280000001192093,
 'eval_top_five': 0.30506667494773865,
 'eval_mrr': 0.22643429040908813,
 'eval_top_ten': 0.437666654586792,
 'eval_runtime': 164.7328,
 'eval_samples_per_second': 91.057,
 'eval_steps_per_second': 5.694,
 'epoch': 3.0}

In [ ]:
### Language Model Classifier Finetuning Curves

In [ ]:
# TODO: Visualize training and validation losses
with open("/home/abunn/ttmp/ExplorationWithLLMs/source/03_alec_temp_name/finetuned_models/dense_1_8/log_history.json", "r") as fin:
    log_history = json.load(fin)

step, tr_loss, val_loss = [], [], []
acc = []
for epoch in log_history:
    step.append(epoch["epoch"])
    tr_loss.append(epoch["train_loss"])
    val_loss.append(epoch["val_loss"])
    acc.append(epoch["accuracy"])

step, tr_loss, val_loss = np.array(step), np.array(tr_loss), np.array(val_loss)
plt.plot(step, tr_loss, 'k-', label="Train")
plt.scatter(step, tr_loss, c='k')
plt.plot(step, val_loss, 'g-', label="Validation")
plt.scatter(step, val_loss, c='g')
plt.legend()
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.plot(step, acc, 'g-', label="Validation")
plt.scatter(step, acc, c='g')
plt.legend()
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.show()